In [1]:
import numpy as np
from gurobipy import *

In [2]:
with open('UFL.dat') as f:
    dem = np.fromstring(f.readline(), sep=',\t')
    c = np.fromstring(f.readline(), sep=',\t')
    c = c.reshape(len(dem), len(c)//len(dem))
    f = np.fromstring(f.readline(), sep=',\t')

In [3]:
n_clients = dem.shape[0]
print('Demand vector:', dem.shape)
n_facilities = f.shape[0]
print('Fixed-cost vector:', f.shape)
print('Variable-cost matrix:', c.shape)

Demand vector: (45,)
Fixed-cost vector: (40,)
Variable-cost matrix: (45, 40)


In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#     LANGRANGIAN RELAXATION
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Create relaxation problem instance
lr = Model('langrangian_relaxation')

# Add variables
x = lr.addVars(n_clients, n_facilities, vtype=GRB.CONTINUOUS, name='x')
y = lr.addVars(n_facilities, vtype=GRB.BINARY, name='y')
p = lr.addVars(n_clients, vtype=GRB.CONTINUOUS, name='p')
lr.update()

# Add constraints
lr.addConstrs((dem[i] - x.sum(i, '*') == p[i] for i in range(n_clients)), name='l_multiplier_penalty')
lr.addConstrs((x[i,j] <= dem[i]*y[j] for i in range(n_clients) for j in range(n_facilities)), name='disagg_capacity_constr')

# Initialize Lagrangian Multipliers for the assignment contraints
u = np.ones(n_clients) 

# Placeholder for penalties
penalties = np.ones(n_clients)

# Re-optimize until complementary slackness conditions apply
# (for each dualized constraint either its multiplier or penalty must be zero)
tol = 1e-6
s = 1
iter_count = 0

while not (u.any() <= tol) | (penalties.any() <= tol):
    
    # Initialize objective function
    obj_lr = LinExpr()
    for j in range(n_facilities):
        obj_lr += f[j]*y[j]
    for i in range(n_clients):
        obj_lr += u[i]*p[i]
        for j in range(n_facilities):
            obj_lr += c[i][j]*x[i,j]
    
    # Set objective
    lr.setObjective(obj_lr, GRB.MINIMIZE)
    
    # Gurobi param
    lr.setParam('OutputFlag', 0)
    
    # Solve model
    lr.optimize()
    
    # Langrangian Multiplier penalties
    penalties = np.array([p[var].x for var in p])
    
    # Update Langrangian Multiplier
    for i in range(n_clients):
        u[i] = max(u[i] + s*(penalties[i]), 0)
    
    # Increase counter
    iter_count += 1
    
    # Print status
    print('\nIteration:', iter_count, '\nObjective value =', lr.objVal)
    
# Print completion status
print('\n  ##########\n     Done!\n  ##########')

Academic license - for non-commercial use only

Iteration: 1 
Objective value = 3974.0

Iteration: 2 
Objective value = 171859.0

Iteration: 3 
Objective value = 179276.0

Iteration: 4 
Objective value = 179927.0

Iteration: 5 
Objective value = 180171.0

Iteration: 6 
Objective value = 180415.0

Iteration: 7 
Objective value = 180659.0

Iteration: 8 
Objective value = 180903.0

Iteration: 9 
Objective value = 181147.0

Iteration: 10 
Objective value = 181391.0

Iteration: 11 
Objective value = 181635.0

Iteration: 12 
Objective value = 181879.0

Iteration: 13 
Objective value = 182123.0

Iteration: 14 
Objective value = 182367.0

Iteration: 15 
Objective value = 182479.0

Iteration: 16 
Objective value = 182579.0

Iteration: 17 
Objective value = 182679.0

Iteration: 18 
Objective value = 182779.0

Iteration: 19 
Objective value = 182879.0

Iteration: 20 
Objective value = 182919.0

  ##########
     Done!
  ##########


In [5]:
# Write lp file
lr.write('langrangian_relaxation.lp')